In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(inches),Snow(inches)
0,0,Busselton,AU,-33.65,115.33,43.00,84,2,8.19,clear sky,0.00,0
1,1,Hermanus,ZA,-34.42,19.23,54.00,87,100,4.00,moderate rain,3.09,0
2,2,Punta Arenas,CL,-53.15,-70.92,35.60,74,75,23.04,broken clouds,0.00,0
3,3,Mar Del Plata,AR,-38.00,-57.56,50.00,53,1,11.41,clear sky,0.00,0
4,4,Beringovskiy,RU,63.05,179.32,53.67,74,96,2.71,overcast clouds,0.00,0


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain(inches)           float64
Snow(inches)             int64
dtype: object

In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_preference = input("Do you want it to be raining? (yes/no) ")
snow_preference = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

if rain_preference =="yes":
    preferred_cities_df = preferred_cities_df[preferred_cities_df["Rain(inches)"] != 0]
else:
    preferred_cities_df = preferred_cities_df[preferred_cities_df["Rain(inches)"] == 0]
if snow_preference =="yes":
    preferred_cities_df = preferred_cities_df[preferred_cities_df["Snow(inches)"] != 0]
else:
    preferred_cities_df = preferred_cities_df[preferred_cities_df["Snow(inches)"] == 0]
                                              

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(inches),Snow(inches)
14,14,Victoria,HK,22.29,114.16,86.00,74,100,11.99,overcast clouds,0.0,0
20,20,Katsuura,JP,35.13,140.30,75.00,96,100,3.00,overcast clouds,0.0,0
30,30,Jamestown,US,42.10,-79.24,84.20,62,1,4.70,clear sky,0.0,0
37,37,Timmins,CA,48.47,-81.33,84.20,54,75,8.05,broken clouds,0.0,0
41,41,Avarua,CK,-21.21,-159.78,75.20,73,63,13.87,broken clouds,0.0,0
42,42,Georgetown,MY,5.41,100.34,82.40,88,20,2.24,few clouds,0.0,0
44,44,East London,ZA,-33.02,27.91,77.94,33,98,14.23,overcast clouds,0.0,0
51,51,Maumere,ID,-8.62,122.21,75.83,78,58,3.49,broken clouds,0.0,0
57,57,Ponta Do Sol,PT,32.67,-17.10,75.20,69,20,6.93,few clouds,0.0,0
60,60,Kudahuvadhoo,MV,2.67,72.89,84.22,73,100,6.93,overcast clouds,0.0,0


In [7]:
preferred_cities_df.count()

City_ID                185
City                   185
Country                185
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
Rain(inches)           185
Snow(inches)           185
dtype: int64

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,Victoria,HK,86.00,overcast clouds,22.29,114.16,
20,Katsuura,JP,75.00,overcast clouds,35.13,140.30,
30,Jamestown,US,84.20,clear sky,42.10,-79.24,
37,Timmins,CA,84.20,broken clouds,48.47,-81.33,
41,Avarua,CK,75.20,broken clouds,-21.21,-159.78,
42,Georgetown,MY,82.40,few clouds,5.41,100.34,
44,East London,ZA,77.94,overcast clouds,-33.02,27.91,
51,Maumere,ID,75.83,broken clouds,-8.62,122.21,
57,Ponta Do Sol,PT,75.20,few clouds,32.67,-17.10,
60,Kudahuvadhoo,MV,84.22,overcast clouds,2.67,72.89,


In [9]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,Victoria,HK,86.00,overcast clouds,22.29,114.16,Mini Hotel Central
20,Katsuura,JP,75.00,overcast clouds,35.13,140.30,Katsuura Hotel Mikazuki
30,Jamestown,US,84.20,clear sky,42.10,-79.24,Hampton Inn & Suites Jamestown
37,Timmins,CA,84.20,broken clouds,48.47,-81.33,The Senator Hotel & Conference Centre
41,Avarua,CK,75.20,broken clouds,-21.21,-159.78,Paradise Inn
42,Georgetown,MY,82.40,few clouds,5.41,100.34,Cititel Penang
44,East London,ZA,77.94,overcast clouds,-33.02,27.91,Tu Casa
51,Maumere,ID,75.83,broken clouds,-8.62,122.21,Sylvia Hotel
57,Ponta Do Sol,PT,75.20,few clouds,32.67,-17.10,Hotel do Campo
60,Kudahuvadhoo,MV,84.22,overcast clouds,2.67,72.89,Kudahuvadhoo Retreat


In [13]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [30]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [31]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")